In [1]:
import datetime
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import date
from datetime import datetime
from newspaper import Article

In [5]:
# Initialize a DataFrame to store news data
news_data = pd.DataFrame(columns=['Date', 'Source', 'Headline', 'Link', 'Content'])

# Define the range of years for data collection
years = range(2013, 2024)

In [7]:
for i, year in enumerate(years):
    print(f'{i}th year')
    # Set the start date for each year
    start_date = datetime(year, 1, 1).date()
    end_date = datetime(year, 12, 31).date()
    current_date = date.today()

    # Number of pages to scrape for each year
    pages = 50

    for page_num in range(1, pages + 1):
        # Check if current date is within the year range
        if current_date > start_date and current_date <= end_date:
            page_url = f'https://in.investing.com/equities/facebook-inc-news/{page_num}'
            try:
                response = requests.get(page_url)
                # Skip the page if server error occurs
                if response.status_code == 502:
                    continue

                # Parse the HTML content of the page
                soup = BeautifulSoup(response.content, 'html.parser')
                articles = soup.find_all('li', {'class': "common-articles-item js-article-item"})

                # Iterate over each article in the page
                for article_tag in articles:
                    time_tag = article_tag.find('time')
                    # Extract the date and convert it to a readable format
                    if time_tag and 'datetime' in time_tag.attrs:
                        article_date_text = time_tag['datetime']
                        article_date = datetime.fromtimestamp(int(article_date_text)).strftime("%b %d, %Y %H:%M")
                    else:
                        article_date = 'Unknown Date'
                    # Extract article source, headline, and link
                    source = article_tag.find('ul', attrs={'class': "details-list"}).find('li').text
                    headline = article_tag.find('h3', attrs={'class': "title"})['title']
                    link = 'https://in.investing.com' + article_tag.find('h3', attrs={'class': "title"}).find('a')['href']

                    # Download and parse the article content
                    article_instance = Article(link)
                    article_instance.download()
                    article_instance.parse()
                    content = article_instance.text

                    # Create a row of data for the article
                    news_row = {'Date': article_date, 'Source': source, 'Headline': headline, 'Link': link, 'Content': content}
                    # Add the row to the DataFrame
                    news_data = pd.concat([news_data, pd.DataFrame([news_row])], ignore_index=True)

            except requests.exceptions.RequestException as e:
                print(f"Error on page {page_num}: {e}")

    # Reset the index of the DataFrame after each year
    news_data.reset_index(drop=True, inplace=True)

# Export the data to a CSV file (optional)
news_data.to_csv('news_data.csv', index=False)

0th year
1th year
2th year
3th year
4th year
5th year
6th year
7th year
8th year
9th year
10th year
